# Hybrid Quantum Optimization for Portfolio Selection
*By [Your Name] | Date: [Insert Date]*

## Business Framing
Portfolio selection is a core task in finance: investors aim to choose assets to maximize **returns** while minimizing **risk** (volatility). Classical methods use *Modern Portfolio Theory (MPT)*, which solves this as a mean-variance optimization problem. Quantum algorithms like QAOA (Quantum Approximate Optimization Algorithm) may offer faster solutions, especially for large portfolios.

### Problem Statement
Given `n` assets with expected returns μ and covariance matrix Σ, find optimal portfolio weights w to maximize:

$$Objective = μ^T w - γ w^T Σ w$$

We compare classical continuous optimization (MPT) with quantum discrete optimization (QAOA).

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install qiskit qiskit_optimization matplotlib numpy scipy

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from qiskit import Aer
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.visualization import plot_histogram

## Classical Portfolio Optimization Baseline

In [ ]:
np.random.seed(42)
n_assets = 5
mu = np.random.uniform(0.05, 0.15, size=n_assets)
sigma_low = np.random.rand(n_assets, n_assets)
sigma = sigma_low @ sigma_low.T
print("Expected Returns:", mu)
print("Covariance Matrix:\n", sigma)

In [ ]:
def classical_objective(w, mu, sigma, gamma):
    return -(np.dot(w, mu) - gamma * np.dot(w, np.dot(sigma, w)))

constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
bounds = tuple((0, 1) for _ in range(n_assets))
gamma_classical = 1.0
initial_guess = np.ones(n_assets) / n_assets
result = minimize(classical_objective, initial_guess, args=(mu, sigma, gamma_classical),
                  method='SLSQP', constraints=constraints, bounds=bounds)
w_classical = result.x
print("Classical Optimal Weights:", w_classical)

## Quantum Approach: QAOA for Portfolio Optimization

In [ ]:
qp = QuadraticProgram()
x = qp.binary_var_list(n_assets, name='x')
qp.minimize(linear=[-mu[i] for i in range(n_assets)], quadratic={(i,j): sigma[i,j] for i in range(n_assets) for j in range(i+1,n_assets)})
print("QUBO Problem:")
qp.print()

In [ ]:
operator = qp.to_operator()
optimizer = COBYLA(maxiter=100)
qaoa = QAOA(optimizer=optimizer, reps=3)
meo = MinimumEigenOptimizer(qaoa)
result_qaoa = meo.solve(qp)
print("Quantum Solution:", result_qaoa)

## Results Comparison

In [ ]:
print("Classical Weights:", w_classical)
print("Quantum Solution:", result_qaoa.x)